# The Backwards One Body Waveform: "BOB"

## Author: Siddharth Mahesh

## This module documents the Backwards One Body waveform calibrated to numerical fits from pySEOBNR's SEOBNRv5HM gravitational waveform approximant.


**Notebook Status:** <font color='red'><b> In Progress? </b></font>

**Validation Notes:** 

### NRPy+ Source Code for this module: [v5HM-BOB_unoptimized_merger_ringdown.py](./Radiation/v5HM-BOB_unoptimized_merger_ringdown.py)

<a id='intro'></a>

## Introduction
$$\label{intro}$$

### The Physical System of Interest

Consider two black holes with masses $m_{1}$, $m_{2}$ and spins ${S}_{1}$, ${S}_{2}$ that are aligned (i.e parallel or antiparallel with respect to each other) in a binary system.  The Backwards One Body ("BOB") waveform $h^{\rm BOB}$ (defined in [this cell](#hBOB)) describes the gravitational wave radiation in the late-inspiral and merger-ringdown stages of this system; we will define $h^{\rm BOB}$ as in [McWilliams,S.T(2019)](https://arxiv.org/pdf/1810.00040.pdf).  There, $h^{\rm BOB}$ is described in terms of the characteristics of final merged black hole. Here, we seek to break up $h^{\rm BOB}$ and document the terms in such a way that the resulting Python code can be used to numerically evaluate it.

Please note that throughout this notebook we adopt the following convention $G = M = c = 1$, where $G$ is the universal gravitational constant, $M = m_1 + m_2$ is the total mass of the binary (**NOT** the mass of the final black hole $M_f$) and, $c$ is the speed of light in vacuum.

Running this notebook to completion will generate a file called v5HM-BOB_unoptimized_merger_ringdown.py. This file contains the Python function v5HM-BOB_unoptimized_merger_ringdown(), which takes as input the initial mass m1, m2 (normalized by the total mass, i.e, $m_! + m_2 = 1$), the values of the normalized spins $\chi_{1,2} = \frac{S_{1,2}}{m_{1,2}^2}$, values for the final mass $M_f$, final spin $\chi_f$, peak waveform frequency $\omega_{22}^{\rm peak}$ .

### Citations
Throughout this module, we will refer to
* [McWilliams,S.T(2019)](https://arxiv.org/pdf/1810.00040.pdf) as BOB
* [Pompili, Buonanno, et al (2023)](https://arxiv.org/pdf/2303.18039.pdf) as PB2023,
* [SEOBNRv5HM-Notes](https://dcc.ligo.org/public/0186/T2300060/002/SEOBNRv5HM.pdf) as V5HM,
* [pySEOBNR documentation/code](https://git.ligo.org/waveforms/software/pyseobnr/) as pySEOBNR.

<a id='outputcreation'></a>

# Step 0: Creating the output directory for SEOBNR \[Back to [top](#toc)\]
$$\label{outputcreation}$$

First we create the output directory for SEOBNR (if it does not already exist):

In [1]:
import sys#Add sys to get cmdline_helper from NRPy top directory; remove this line and next when debugged
sys.path.append('../')
import cmdline_helper as cmd     # NRPy+: Multi-platform Python command-line interface

# Create C code output directory:
Ccodesdir = "Radiation"
# Then create an output directory in case it does not exist
cmd.mkdir(Ccodesdir)

<a id='hBOB'></a>

# Step : The BOB waveform polarizations $h^{\textrm{BOB}}$ \[Back to [top](#toc)\]
$$\label{hBOB}$$

The BOB waveform polarizations $h^{\rm BOB}$ are given as:

\begin{equation*}
    h^{\rm BOB}_{+} = h\cos{\phi}\\
    h^{\rm BOB}_{\times} = h\sin{\phi}.
\end{equation*}

Here $h$ (defined in [this cell](#h)) is the strain amplitude, and $\phi$ (defined in [this cell](#phi)) is the waveform phase.

In [2]:
%%writefile $Ccodesdir/v5HM-BOB_merger_ringdown_on_top.txt
hplus = h*sp.cos(phi)
hcross = h*sp.sin(phi)

Overwriting Hamiltonian/v5HM_Hamiltonian-Hreal_on_top.txt


<a id='h'></a>

# Step : The strain amplitude $h$ \[Back to [top](#toc)\]
$$\label{h}$$

The strain amplitude as stated in the discussion below Equation 5 of [BOB](https://arxiv.org/pdf/1810.00040.pdf) :

\begin{equation*}
    h = \frac{A_p}{4\Omega^2}\textrm{sech}\left(\frac{t - t_p}{\tau}\right)
\end{equation*}

Here, $A_p$ is defined in [this cell](#ap), $t_{p}$ is defined in [this cell](#tp), $\Omega$ is defined in [this cell](#omega). The time $t$ and the QNM damping timescale $\tau$ are given as inputs. 

In [3]:
%%writefile -a $Ccodesdir/v5HM-BOB_merger_ringdown_on_top.txt
h = (Ap/4/Omega2)*sp.sech((t - tp)/tau)

Appending to Hamiltonian/v5HM_Hamiltonian-Hreal_on_top.txt


<a id='phi'></a>

# Step : The waveform phase $\phi$ \[Back to [top](#toc)\]
$$\label{\phi}$$

The waveform phase is given in terms of the orbital phase which is given by equation 10 of [BOB](https://arxiv.org/pdf/1810.00040.pdf):
\begin{equation*}
    \phi = 2\Phi\\
    \Phi = \textrm{arctan}_{+} + \textrm{arctanh}_{+} - \textrm{arctan}_{-} - \textrm{arctanh}_{-}.
\end{equation*}

Here, $\textrm{arctan}[\textrm{h}]_{\pm}$ is defined in [this cell](#arctanhpm). 

In [4]:
%%writefile -a $Ccodesdir/v5HM-BOB_merger_ringdown_on_top.txt
phi = 2*Phi
Phi = arctanp + arctanhp - arctanm - arctanhm

Appending to Hamiltonian/v5HM_Hamiltonian-Hreal_on_top.txt


<a id='arctanhpm'></a>

# Step : ${\rm arctan}[{\rm h}]_{\pm}$ \[Back to [top](#toc)\]
$$\label{arctanhpm}$$

The phase terms are described in Equation 10 of [BOB](https://arxiv.org/pdf/1810.00040.pdf):
\begin{equation*}
    {\rm arctan}[{\rm h}]_{\pm} = \kappa_{\pm}\tau\left[ {\rm arctan}[{\rm h}]\left(\frac{\Omega}{\kappa_{\pm}}\right) - {\rm arctan}[{\rm h}]\left(\frac{\Omega_0}{\kappa_{\pm}}\right) \right]
\end{equation*}

Here,

Note: due to some errors in sympy's handling of inverse hyperbolic tangents, we choose the alternative definition

$$
    {\rm arctanh}(x) \equiv \frac{1}{2}\ln\left(\frac{1 + x}{1 - x}\right)
$$

In [7]:
%%writefile -a $Ccodesdir/v5HM-BOB_merger_ringdown_on_top.txt
arctanp = kappap*tau*( sp.atan(Omega/kappap) - sp.atan(Omega0/kappap) )
arctanm = kappam*tau*( sp.atan(Omega/kappam) - sp.atan(Omega0/kappam) )
arctanhp = kappap*tau*( atanh_Omega_kappap - atanh_Omega0_kappap )
arctanhm = kappam*tau*( atanh_Omega_kappam - atanh_Omega0_kappam )
atanh_Omega_kappap = sp.Rational(1,2)*sp.log( (1 + Omega/kappap)/(1 - Omega/kappap) )
atanh_Omega0_kappap = sp.Rational(1,2)*sp.log( (1 + Omega0/kappap)/(1 - Omega0/kappap) )
atanh_Omega_kappam = sp.Rational(1,2)*sp.log( (1 + Omega/kappam)/(1 - Omega/kappam) )
atanh_Omega0_kappam = sp.Rational(1,2)*sp.log( (1 + Omega0/kappam)/(1 - Omega0/kappam) )

Appending to Hamiltonian/v5HM_Hamiltonian-Hreal_on_top.txt


<a id='omega'></a>

# Step : The Effective Orbital Frequency $\Omega$ \[Back to [top](#toc)\]
$$\label{omega}$$

The effective orbital frequency is given defined in Equation 7 of [BOB](https://arxiv.org/pdf/1810.00040.pdf):

\begin{equation*}
    \Omega = \left\{ \Omega_0^4 + k\left[ \tanh\left(\frac{t - t_p}{\tau}\right) - \tanh\left(\frac{t_0 - t_p}{\tau}\right) \right] \right\}^{1/4}.
\end{equation*}

Where, the reference frequency $\Omega_0$ is defined in [this cell](#omega0), $k$ is defined in [this cell], and $t_p$ is defined in [this cell](tp). The times $t$, $t_0$, and the QNM damping timescale $\tau$ are given as inputs. 

In [6]:
%%writefile -a $Ccodesdir/v5HM-BOB_merger_ringdown_on_top.txt
Omega = ( Omega0**4 + k*( sp.tanh((t - tp)/tau) - sp.tanh((t0 - tp)/tau) ) )**(1/4)

Appending to Hamiltonian/v5HM_Hamiltonian-Hreal_on_top.txt


<a id='kappapm'></a>

# Step : $\kappa_{\pm}$ \[Back to [top](#toc)\]
$$\label{kappapm}$$

The coefficient $\kappa_{\pm}$ is expressed in Equation 10 of [BOB](https://arxiv.org/pdf/1810.00040.pdf):
\begin{equation*}
    \kappa_{\pm} = \left\{ \Omega_0^4 \pm k\left[ 1 \mp \tanh\left(\frac{t_0 - t_p}{\tau}\right) \right] \right\}^{1/4}.
\end{equation*}

Here, $\Omega_0$ is defined in [this cell](#omega0), $t_p$ is defined in [this cell](#tp) and $k$ is defined [in this cell](#k).

In [8]:
%%writefile -a $Ccodesdir/v5HM-BOB_merger_ringdown_on_top.txt
kappap = (Omega0**4 + k*( 1 - sp.tanh((t0 - tp)/tau) ))**(1/4)

Appending to Hamiltonian/v5HM_Hamiltonian-Hreal_on_top.txt


<a id='k'></a>

# Step : $k$ \[Back to [top](#toc)\]
$$\label{bkerrnpa}$$

The coefficient $k$ is expressed in Equation 8 of [BOB](https://arxiv.org/pdf/1810.00040.pdf):
\begin{equation*}
    k = \left( \frac{\Omega_{\rm QNM}^4 - \Omega_0^4}{1 - \tanh\left[(t_0 - t_p)/\tau\right]} \right).
\end{equation*}

Here, $\Omega_0$ is defined in [this cell](#omega0), $t_p$ is defined in [this cell](#t_p). The reference time $t_0$, the QNM frequency $\Omega_{\rm QNM}$, and the QNM damping time $\tau$ are given as inputs.

In [9]:
%%writefile -a $Ccodesdir/v5HM-BOB_merger_ringdown_on_top.txt
k = (OmegaQNM**4 - Omega0**4)/(1 - sp.tanh((t0 - tp)/tau))

Appending to Hamiltonian/v5HM_Hamiltonian-Hreal_on_top.txt


<a id='ap'></a>

# Step : $A_p$ \[Back to [top](#toc)\]
$$\label{ap}$$

The peak $\psi_4$ amplitude is calculated by matching the strain amplitude $h$ at the peak strain time to it's value given by numerical relativity fits:
\begin{equation*}
    A_p = h_{\rm NR}\omega^2_{\rm NR}\cosh\left( \frac{t_0 - t_p}{\tau} \right).
\end{equation*}

Here, $h_{\rm NR}$ and $\omega_{rm NR}$ are the peak strain ampitude and the corresponding frequency as determined by numerical relativity fits. $t_0$ and $\tau$ are given as inputs and $t_p$ is defined in [this cell](#tp).

In [10]:
%%writefile -a $Ccodesdir/v5HM-BOB_merger_ringdown_on_top.txt
Ap = hNR*(omegaNR**2)*sp.cosh((t0 - tp)/tau)

Appending to Hamiltonian/v5HM_Hamiltonian-Hreal_on_top.txt


<a id='tp'></a>

# Step : $t_p$ \[Back to [top](#toc)\]
$$\label{tp}$$

The peak $\psi_4$ time $t_p$ is obtained by solving for it under the condition that the reference time corresponds to the peak of the strain amplitude $\dot{h}(t_0) = 0$ to give:
\begin{equation*}
    t_p = t_0 - 2\tau\ln\left(\frac{\Omega_0}{\Omega_{\rm QNM}}\right).
\end{equation*}

Here, $\Omega_0$ is defined in [this cell](#omega0). The QNM frequency $\Omega_{\rm QNM}$, the QNM damping time $\tau$, and the reference time $t_0$ are given as inputs.

In [12]:
%%writefile -a $Ccodesdir/v5HM-BOB_merger_ringdown_on_top.txt
tp = t0 - 2*tau*sp.log(Omega0/OmegaQNM)

Appending to Hamiltonian/v5HM_Hamiltonian-Hreal_on_top.txt


<a id='omega0'></a>

# Step : $\Omega_0$ \[Back to [top](#toc)\]
$$\label{omega0}$$

The reference effective orbital frequency $\Omega_{0}$ is expressed in terms of the waveform frequency corresponding to the peak of the strain amplitude:
\begin{equation*}
    \Omega_0 = \frac{\omega_{\rm NR}}{2}.
\end{equation*}

Here, $\omega_{\rm NR}$ is given as an input.

In [11]:
%%writefile -a $Ccodesdir/v5HM-BOB_merger_ringdown_on_top.txt
Omega0 = omegaNR/2

Appending to Hamiltonian/v5HM_Hamiltonian-Hreal_on_top.txt


# Step : Saving the expressions

Up till now, the expressions required for the Hamiltonian have been stored in a .txt file. for the sake of readability, some of the expressions have been written in more than one line. 

Therefore, we save the expressions in one-line format to parse into sympy for generating optimized code or derivative expressions.

In [27]:
import os 
with open(os.path.join(Ccodesdir,"v5HM_Hamiltonian-Hreal_on_top-oneline.txt"), "w") as output:
    count = 0
    # Read output of this notebook
    for line in list(open(os.path.join(Ccodesdir,"v5HM_Hamiltonian-Hreal_on_top.txt"),"r")):
        # Read the first line
        if count == 0:
            prevline=line
        #Check if prevline is a complete expression
        elif "=" in prevline and "=" in line:
            output.write("%s\n" % prevline.strip('\n'))
            prevline=line
        # Check if line needs to be adjoined to prevline
        elif "=" in prevline and not "=" in line:
            prevline = prevline.strip('\n')
            prevline = (prevline+line).replace(" ","")
        # Be sure to print the last line.
        if count == len(list(open(os.path.join(Ccodesdir,"v5HM_Hamiltonian-Hreal_on_top.txt"))))-1:
            if not "=" in line:
                print("ERROR. Algorithm not robust if there is no equals sign on the final line. Sorry.")
                sys.exit(1)
            else:
                output.write("%s" % line)
        count = count + 1

with open(os.path.join(Ccodesdir,"v5HM_Hamiltonian-Hreal_on_bottom.txt"), "w") as output:
    for line in reversed(list(open(os.path.join(Ccodesdir,"v5HM_Hamiltonian-Hreal_on_top-oneline.txt"),"r"))):
        output.write("%s\n"%line.rstrip())


# Generate unoptimized python code

In this step, we'll simply store the Hamiltonian expressions in a python module as is in order to run preliminary validation tests. In addition, we will compare the results of our Hamiltonian function with that of the pyseobnr Hamiltonian function.

In [28]:
with open(os.path.join(Ccodesdir,"v5HM_unoptimized_hamiltonian.py"), "w") as output:
    output.write("import numpy as np\ndef v5HM_unoptimized_hamiltonian(m1, m2, r, prstar, pphi, chi1, chi2,verbose = False):\n")
    for line in reversed(list(open(os.path.join(Ccodesdir,"v5HM_Hamiltonian-Hreal_on_top-oneline.txt"),"r"))):
        output.write("    %s\n" % line.rstrip().replace("sp.sqrt", "np.sqrt").replace("sp.Rational",
                                "np.divide").replace("sp.log","np.log").replace("sp.pi","np.pi").replace("sp.EulerGamma","np.euler_gamma"))
    output.write("    if not verbose:\n        return Hreal,xi\n    else:\n        return Hreal,xi,Aalign,Balignnp,Bkerreqnp,Qalign,Heven,Hodd,QalignSS,Qnos,Galigna3,gam,gap,SOcalib,u,nu,ap,am,r,prstar,pphi,chi1,chi2,m1,m2")
    
import Hamiltonian.v5HM_unoptimized_hamiltonian as Hreal_unopt
import Hamiltonian.pyseobnr_hamiltonian as Hreal_true
import numpy as np

#generate random input data
gt_pert_total = []
gt_pert_O1 = []
gt_pert_O2 = []
gt_pert_gtO3 = []
N = 10000
rng = np.random.default_rng(seed = 50)
nu = rng.random(N)*.25
chi1 = 2.*rng.random(N)-1.
chi2 = 2.*rng.random(N)-1.
m2 = (1 - np.sqrt(1 - 4*nu))*.5
m1 = (1 + np.sqrt(1 - 4*nu))*.5
r = rng.random(N)*17 + 3.
phi = rng.random(N)*2.*np.pi
prstar = rng.random(N)*20. - 10.
pphi = rng.random(N)*20. - 10.

# generate randomized perturbation in order to find the appropriate tolerance
pert_exponent = 1e-14;
pert_sign = 2*rng.integers(0,high = 2,size = N) - 1;
pert_mantissa = 3.*rng.random(N) + 1. ;
pert = 1. + pert_sign*pert_mantissa*pert_exponent
chi1pert = chi1*pert 
chi2pert = chi2*pert
m2pert = m2*pert
m1pert = m1*pert
rpert = r*pert
phipert = phi*pert
prstarpert = prstar*pert
pphipert = pphi*pert

nans = []
def E_rel(a,b):
    return np.abs((a - b)/a)

for i in range(N):
    q  = [r[i],phi[i]]
    p = [prstar[i],pphi[i]]
    qpert  = [rpert[i],phipert[i]]
    ppert = [prstarpert[i],pphipert[i]]
    H_unopt,xi = Hreal_unopt.v5HM_unoptimized_hamiltonian(m1[i],m2[i],r[i],prstar[i],pphi[i],chi1[i],chi2[i],verbose = False)
    H_pyseobnr,xi = Hreal_true.evaluate_H(q,p, chi1[i], chi2[i], m1[i], m2[i],verbose = False)
    H_pert,xi = Hreal_true.evaluate_H(qpert,ppert, chi1pert[i], chi2pert[i], m1pert[i], m2pert[i],verbose = False)
    e_rel = E_rel(H_pyseobnr,H_unopt)
    tol = E_rel(H_pyseobnr,H_pert)
    if (np.isnan(H_pyseobnr) and np.isnan(H_unopt) and np.isnan(H_pert)):
        nans.append(i)
    else:
        if e_rel>tol:
            gt_pert_total.append(i)
            if e_rel > 1000*tol:
                gt_pert_gtO3.append(i)
            elif e_rel > 100*tol:
                gt_pert_O2.append(i)
            elif e_rel > 10*tol:
                gt_pert_O1.append(i)

print("Of total ",str(N)," comparisons,\n",
     "number of cases with relative error (total)  greater than allowed: ",len(gt_pert_total),"\n",
     "number of cases with relative error O(10)    greater than allowed: ",len(gt_pert_O1),"\n",
     "number of cases with relative error O(100)   greater than allowed: ",len(gt_pert_O2),"\n",
     "number of cases with relative error O(1000+) greater than allowed: ",len(gt_pert_gtO3))



Of total  10000  comparisons,
 number of cases with relative error (total)  greater than allowed:  0 
 number of cases with relative error O(10)    greater than allowed:  0 
 number of cases with relative error O(100)   greater than allowed:  0 
 number of cases with relative error O(1000+) greater than allowed:  0
